In [ ]:
import boto3
import sagemaker

In [ ]:
role = 'arn:aws:iam::638608113287:role/service-role/AmazonSageMaker-ExecutionRole-20180731T132167'
sess = sagemaker.Session()

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [ ]:
image = '638608113287.dkr.ecr.us-east-1.amazonaws.com/faster-rcnn:latest'
train_instance_type = 'ml.p3.16xlarge'
instance_count = 1
output_path="s3://model-artifacts-alkymi/faster-rcnn/"
data_location = "s3://training-data-alkymi/pageseg/2019-07-08_20:04:12.342294"
# data_location = "s3://training-data-alkymi/pageseg/7"
hyperparameters = {
  "batch_size": "64",
  "epochs":"100",
  "lr": "0.0001",
  "lr_decay_gamma": "0.1",
  "lr_patience": "2",
  "patience": "4",
  "imdb_name": "pdfpages",
  "num_workers": "8",
  "USE_FLIPPED": "False"
}

metric_definitions = [
    {'Name': 'train loss', 'Regex': 'train loss: (.*?);'},
    {'Name': 'train rpn_cls', 'Regex': 'train rpn_cls: (.*?);'},
    {'Name': 'train rpn_box', 'Regex': 'train rpn_box: (.*?);'},
    {'Name': 'train rcnn_cls', 'Regex': 'train rcnn_cls: (.*?);'},
    {'Name': 'train rcnn_box', 'Regex': 'train rcnn_box: (.*?);'},
    {'Name': 'val loss', 'Regex': 'val loss: (.*?);'},
    {'Name': 'val rpn_cls', 'Regex': 'val rpn_cls: (.*?);'},
    {'Name': 'val rpn_box', 'Regex': 'val rpn_box: (.*?);'},
    {'Name': 'val rcnn_cls', 'Regex': 'val rcnn_cls: (.*?);'},
    {'Name': 'val rcnn_box', 'Regex': 'val rcnn_box: (.*?);'}]

In [ ]:
estimator = sagemaker.estimator.Estimator(image,
                                          role, instance_count, train_instance_type,
                                          output_path=output_path,
                                          sagemaker_session=sess, 
                                          hyperparameters=hyperparameters,
                                          metric_definitions=metric_definitions,
                                          train_max_run=432000)

In [ ]:
estimator.fit(data_location)

In [ ]:
training_job_name = ''
sagemaker.estimator.Estimator.attach(training_job_name, sagemaker_session=sess)

In [ ]:
deploy_instance_type = 'ml.p2.xlarge'
predictor = estimator.deploy(initial_instance_count=1, instance_type=deploy_instance_type)